In [ ]:
import ee

In [ ]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file='/content/drive/MyDrive/w/private-key.json'
)

ee.Initialize(credentials)

In [ ]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

In [ ]:
datasets_P = mask_nc_file(
    nc = "/content/drive/MyDrive/w/Past/P/Daily/1979001.nc",
    variable = "precipitation",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
datasets_P

In [ ]:
crs = 4326
shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])




soil_era_evaporation = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('1978-12-31', '1979-01-01')).select('volumetric_soil_water_layer_1')

soil_era_transpiration = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('1978-12-31', '1979-01-01')).select('volumetric_soil_water_layer_3')

soil_era_transition = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('1978-12-31', '1979-01-01')).select('volumetric_soil_water_layer_4')


soil_era_evaporation = soil_era_evaporation.map(lambda image: image.clip(shape))
soil_era_transpiration = soil_era_transpiration.map(lambda image: image.clip(shape))
soil_era_transition = soil_era_transition.map(lambda image: image.clip(shape))

soil_era_evaporation = soil_era_evaporation.set("system:time_start", ee.Date("1979-01-01"))
soil_era_transpiration = soil_era_transpiration.set("system:time_start", ee.Date("1979-01-01"))
soil_era_transition = soil_era_transition.set("system:time_start", ee.Date("1979-01-01"))


arr_soil_era_evaporation = soil_era_evaporation.wx.to_xarray(scale=11132, crs=f'EPSG:{crs}')
arr_soil_era_transpiration = soil_era_transpiration.wx.to_xarray(scale=11132, crs=f'EPSG:{crs}')
arr_soil_era_transition = soil_era_transition.wx.to_xarray(scale=11132, crs=f'EPSG:{crs}')

da_soil_era_evaporation = arr_soil_era_evaporation.rio.write_crs(f"epsg:{crs}", inplace=True)
da_soil_era_transpiration = arr_soil_era_transpiration.rio.write_crs(f"epsg:{crs}", inplace=True)
da_soil_era_transition = arr_soil_era_transition.rio.write_crs(f"epsg:{crs}", inplace=True)



datasets_P = datasets_P.rename({'lon': 'x','lat': 'y'})
datasets_P1 = datasets_P.rio.write_crs(f"epsg:{crs}", inplace=True)


xds_repr_match__soil_era_evaporation = da_soil_era_evaporation.rio.reproject_match(datasets_P1, resampling = rasterio.enums.Resampling.nearest)
xds_repr_match__soil_era_transpiration = da_soil_era_transpiration.rio.reproject_match(datasets_P1, resampling = rasterio.enums.Resampling.nearest)
xds_repr_match__soil_era_transition = da_soil_era_transition.rio.reproject_match(datasets_P1, resampling = rasterio.enums.Resampling.nearest)


xds_repr_match__soil_era_evaporation = xds_repr_match__soil_era_evaporation * (70-0)
xds_repr_match__soil_era_transpiration = xds_repr_match__soil_era_transpiration * (1000 - 280)
xds_repr_match__soil_era_transition = xds_repr_match__soil_era_transition * (2890-1000)

xds_repr_match__soil_era_evaporation

In [ ]:
# Convert time coordinate to NumPy datetime64 type
time_values_evap = np.asarray(xds_repr_match__soil_era_evaporation['time'].values, dtype='datetime64[D]')
time_values_transp = np.asarray(xds_repr_match__soil_era_transpiration['time'].values, dtype='datetime64[D]')
time_values_trans = np.asarray(xds_repr_match__soil_era_transition['time'].values, dtype='datetime64[D]')

# Subtract one day from each time value
updated_time_values_evap = time_values_evap + np.timedelta64(1, 'D')
updated_time_values_transp = time_values_transp + np.timedelta64(1, 'D')
updated_time_values_trans = time_values_trans + np.timedelta64(1, 'D')

# Update the time coordinate with the modified values
xds_repr_match__soil_era_evaporation['time'] = updated_time_values_evap
xds_repr_match__soil_era_transpiration['time'] = updated_time_values_transp
xds_repr_match__soil_era_transition['time'] = updated_time_values_trans

In [ ]:
merged_dataset = xr.merge([xds_repr_match__soil_era_evaporation, xds_repr_match__soil_era_transpiration, xds_repr_match__soil_era_transition])

merged_dataset.to_netcdf("/content/drive/MyDrive/w/soil_water_content_1979001.nc")